In [ ]:
# Вычисления
import numpy as np
import pandas as pd
import scipy
from sklearn.mixture import GaussianMixture
# Визуализация
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
%matplotlib notebook

In [ ]:
import lab1

In [ ]:
df = pd.read_csv('data/raifhack_train.csv', usecols=lambda x: x not in ['floor'])

## Модель смеси

In [ ]:
bi_var = df['osm_subway_closest_dist'].sample(frac=0.1, random_state=0)
bi_var = bi_var.apply(np.log)

In [ ]:
fig = px.histogram(bi_var, nbins=50, template='plotly_white')
fig.layout.showlegend=False
fig.show()

In [ ]:
sns.set_style('whitegrid')
plot = sns.histplot(bi_var, kde=True, bins=30, fill=False)
plot.lines[0].set_color('black')
plot

**Количество компонент смотрим по BIC**

In [ ]:
gm_x = bi_var.values.reshape(-1, 1)
gm_search = list(range(1, 9))
bic_scores = [
    GaussianMixture(n_components=n_components, random_state=0).fit(gm_x).bic(gm_x)
    for n_components in gm_search 
]

In [ ]:
fig = px.line(x=gm_search, y=bic_scores, template='plotly_white')
fig.data[0].mode='lines+markers'
fig.update_layout(
    xaxis_title='Число компонент',
    yaxis_title='BIC score',
)
fig.show()

**Посмотрим выделенные кластеры**

In [ ]:
clusters = GaussianMixture(n_components=2, random_state=0).fit_predict(gm_x)

In [ ]:
px.histogram( 
    bi_var,
    nbins=50,
    color=clusters,
    template='plotly_white'
)

In [ ]:
np.unique(clusters, return_counts=True)

## Выбираем признак

In [ ]:
# var = bi_var[clusters==1].sample(150, random_state=0)
var = bi_var[clusters==0].sample(150, random_state=0)

**Преобразования**

In [ ]:
# var = var.apply(np.log)

In [ ]:
var = lab1.cut_tails(var)

In [ ]:
# Определяем количество бинов
nbins = lab1.get_nbins(var)
nbins

## Описательные статистики

In [ ]:
fig = px.histogram(var, 
                   marginal='box', 
                   template='plotly_white')
fig.layout.showlegend=False
fig.show()

In [ ]:
{i:round(j,5) 
 for i,j in var.describe().to_dict().items()}

In [ ]:
{i:round(j,5)
 for i,j in lab1.get_main_quantiles(var).items()}

## Подбор параметров

In [ ]:
# Теоретическое распределение
theory_distr = scipy.stats.norm

In [ ]:
params = theory_distr.fit(var)
params = np.array(params)

# params = np.array([
#     var.mean(),
#     var.std()
# ])

params

**Метод наименьших квадратов**

In [ ]:
ls_result = lab1.least_squares_method(var, theory_distr, x0=params*0.9)

In [ ]:
print(ls_result)

In [ ]:
ls_result.x

**Метод максимального правдоподобия**

In [ ]:
ml_result = lab1.max_likelihood_method(var, theory_distr, x0=params*0.9)

In [ ]:
print(ml_result)

In [ ]:
ml_result.x

In [ ]:
# Готовое распределение
theory_distr_with_params = theory_distr(*params)

## Проверка подобранного распределения

**QQ Plot**

In [ ]:
lab1.qq_plot(var, theory_distr_with_params)

**KDE Plot**

In [ ]:
# lab1.kde_plot(var, theory_distr_with_params)

**Гистограмма**

In [ ]:
lab1.hist_fitted_plot(var, theory_distr_with_params)

## Статистические тесты

**Тест Колмогорова-Смирнова**

In [ ]:
scipy.stats.kstest(var, theory_distr_with_params.cdf)

**Тест Хи-квадрат**

In [ ]:
f_obs, f_exp = lab1.chi_square_bins(var, theory_distr_with_params, nbins)
f_obs, f_exp

In [ ]:
scipy.stats.chisquare(f_obs=f_obs, f_exp=f_exp)

**Тест Жака-Бера**

In [ ]:
if theory_distr.name == 'norm':
    print( scipy.stats.jarque_bera(var) )

## Сэмплирование

In [ ]:
# sample_red = lab1.clt_generator(size=10_000, n_layers=100, mean=params[0], std=params[1])

In [ ]:
# sample_blue = lab1.clt_generator(size=10_000, n_layers=100, mean=params[0], std=params[1])

In [ ]:
sample = np.concatenate([
    sample_red, sample_blue
])

In [ ]:
bi_var_sample = np.concatenate([
    bi_var[clusters==1].sample(150, random_state=0),
    bi_var[clusters==0].sample(150, random_state=0)
])

In [ ]:
fig = px.histogram(sample,
                   nbins=100,
                   marginal='box', 
                   template='plotly_white')
fig.layout.showlegend=False
fig.show()

**QQ Plot**

In [ ]:
uniform_quantiles = np.linspace(0, 1, 50+1)[1:-1]
fig = px.scatter(
    x=np.quantile(bi_var_sample, uniform_quantiles),
    y=np.quantile(sample, uniform_quantiles),
    template='plotly_white'
)

fig.add_trace(go.Scatter(
    x=np.quantile(sample, uniform_quantiles),
    y=np.quantile(sample, uniform_quantiles),
    line_color='lightgrey',
    line_dash='dash'
))

fig.data[0].marker.size = 5
fig.layout.showlegend=False

fig.update_layout(
    xaxis_title='Фактические значения', 
    yaxis_title='Теоретические значения'
)

fig.show()